In [1]:
import pandas as pd
import nltk
import string
import spacy

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\furdu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
df = pd.read_csv('factnews_dataset.csv')

import pandas as pd
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

# Baixe as stopwords uma vez (se ainda não baixou)
nltk.download('stopwords')

def analise(df, text_col='sentences', class_col='classe', top_n=10, lang='portuguese'):
    """
    Analisa as top_n palavras mais frequentes para cada classe, excluindo stop words.

    Args:
        df (pd.DataFrame): DataFrame com colunas de texto e classe.
        text_col (str): Nome da coluna de texto.
        class_col (str): Nome da coluna da classe.
        top_n (int): Quantidade de palavras mais frequentes a retornar.
        lang (str): Idioma das stopwords, ex: 'portuguese' ou 'english'.
        
    Returns:
        dict: Dicionário com classes como chaves e listas das palavras + frequências como valores.
    """

    stop_words = set(stopwords.words(lang))

    def tokenize(text):
        text = text.lower()
        words = re.findall(r'\b\w+\b', text)
        words = [w for w in words if w not in stop_words]
        return words
        
    resultado = {}
    classes = df[class_col].unique()
    
    for c in classes:
        textos = df[df[class_col] == c][text_col].dropna()
        palavras = []
        for texto in textos:
            palavras.extend(tokenize(texto))
        contagem = Counter(palavras)
        resultado[c] = contagem  # GUARDE AQUI O COUNTER, NÃO A LISTA
        print(f"Classe {c}: {contagem.most_common(top_n)}")

    if 0 in resultado and 1 in resultado:
        conjunto_0 = set(resultado[0].keys())
        conjunto_1 = set(resultado[1].keys())
        unicas_0 = conjunto_0 - conjunto_1
        print(f"Palavras únicas na classe 0 (não em 1): {sorted(list(unicas_0))}")
    if 0 in resultado and 1 in resultado:
        conjunto_0 = set(resultado[0].keys())
        conjunto_1 = set(resultado[1].keys())
        unicas_1 = conjunto_1 - conjunto_0
        print(f"Palavras únicas na classe 1 (não em 0): {sorted(list(unicas_1))}")
    return resultado

# Exemplo de uso
df = pd.read_csv('factnews_dataset.csv')
top_palavras = analise(df, text_col='sentences', class_col='classe', top_n=10)
print(top_palavras)
    

Classe -1: [('disse', 170), ('presidente', 113), ('afirmou', 96), ('lula', 81), ('bolsonaro', 69), ('sobre', 68), ('pessoas', 53), ('ter', 53), ('brasil', 50), ('porque', 48)]
Classe 0: [('presidente', 461), ('lula', 318), ('segundo', 283), ('feira', 274), ('bolsonaro', 266), ('r', 257), ('sobre', 202), ('nesta', 195), ('contra', 186), ('ex', 185)]
Classe 1: [('presidente', 55), ('lula', 53), ('bolsonaro', 53), ('sobre', 36), ('ex', 35), ('governo', 33), ('feira', 26), ('brasil', 23), ('segundo', 22), ('anos', 22)]
Palavras únicas na classe 0 (não em 1): ['00', '000', '000m', '002', '007', '01', '01542', '01682', '026', '02h', '03', '032', '04', '043', '04371', '04944', '050', '058', '06', '062', '07', '07340', '075', '08', '09', '100', '101', '104', '106', '1061', '10h', '10h13', '10h30', '10ª', '11', '110', '113', '115', '117', '118', '119º', '11h30', '11h40', '11h40m', '120', '122', '124', '126', '127', '128', '12h30', '12min', '130', '133', '136', '139', '13h06', '13ª', '13º', '140

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\furdu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def preprocess(text):
    tokens = nltk.word_tokenize(str(text).lower())
    tokens = [t for t in tokens if t not in string.punctuation]
    return tokens

ver verbo verbo numa frase - prob um bias

In [ ]:
import pandas as pd
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

def citacao(text):
    return '"' in str(text) or '“' in str(text) or '”' in str(text) or '[' in str(text) 

def bias(text):
    return '?' in str(text)

def classificar(row):
    if citacao(row['sentences']):
        return -1  
    if bias(row['sentences']):
        return 1
    return 0

def analise(df, text_col='sentences', class_col='classe', top_n=10, lang='portuguese'):
    stop_words = set(stopwords.words(lang))

    def tokenize(text):
        text = text.lower()
        words = re.findall(r'\b\w+\b', text)
        words = [w for w in words if w not in stop_words]
        return words
    
    resultado = {}
    classes = df[class_col].unique()
    
    for c in classes:
        textos = df[df[class_col] == c][text_col].dropna()
        palavras = []
        for texto in textos:
            palavras.extend(tokenize(texto))
        contagem = Counter(palavras)
        resultado[c] = contagem
        print(f"Classe {c}: {contagem.most_common(top_n)}")
    return resultado

def imprimir_unicas(resultado, classe_base=0, classes_comparar=[-1, 1]):
    """
    Imprime palavras únicas nas classes de interesse comparadas à classe_base.
    """
    if classe_base not in resultado:
        print(f"Classe base {classe_base} não encontrada no resultado.")
        return
    
    base_palavras = set(resultado[classe_base].keys())
    
    for c in classes_comparar:
        if c in resultado:
            comparar_palavras = set(resultado[c].keys())
            unicas = comparar_palavras - base_palavras
            print(f"Palavras únicas na classe {c} (não na classe {classe_base}): {sorted(list(unicas))}")
        else:
            print(f"Classe {c} não encontrada no resultado.")
#teste
# Exemplo de uso:
df = pd.read_csv('factnews_dataset.csv')
a= 1
# Aplicar classificação
df['classe'] = df.apply(classificar, axis=1)

# Obter análise de palavras
resultado = analise(df, text_col='sentences', class_col='classe', top_n=10)

# Mostrar palavras únicas para classes -1 e 1 em relação a 0
imprimir_unicas(resultado)


Classe -1: [('disse', 191), ('presidente', 139), ('afirmou', 111), ('lula', 87), ('sobre', 75), ('bolsonaro', 74), ('ter', 55), ('segundo', 54), ('contra', 47), ('brasil', 46)]
Classe 0: [('presidente', 479), ('lula', 358), ('bolsonaro', 308), ('feira', 279), ('segundo', 276), ('r', 259), ('sobre', 222), ('governo', 214), ('ex', 213), ('nesta', 192)]
Classe 1: [('disse', 14), ('presidente', 11), ('sobre', 9), ('lula', 7), ('bolsonaro', 6), ('casa', 6), ('pessoas', 6), ('vida', 6), ('estado', 6), ('direito', 6)]
Palavras únicas na classe -1 (não na classe 0): ['1503', '1506', '15ª', '16ª', '1846', '1932', '1963', '1970', '1980', '220', '2o', '305', '40min', '462', '515', '5h', '830', 'abandonou', 'abjeto', 'abominável', 'abordar', 'absorção', 'abstenha', 'abster', 'abstivesse', 'absurdas', 'absurdo', 'acabe', 'acatado', 'acelerado', 'acentuação', 'acentuou', 'acertei', 'acertos', 'acessando', 'achei', 'acinte', 'acionária', 'acometendo', 'acompanhamento', 'acompanhamentos', 'acontecia',

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\furdu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
df['Previsto'] = df.apply(classificar, axis=1)

In [36]:
print(df[['sentences', 'classe', 'Previsto']].head())

                                           sentences  classe  Previsto
0            "Não houve sobreviventes", disse Okala.      -1        -1
1  As vítimas do acidente foram 14 passageiros e ...       0         0
2  O avião explodiu e se incendiou, acrescentou o...       0         0
3  O porta-voz informou que o avião, um Soviet An...       0         0
4  Segundo fontes aeroportuárias, os membros da t...       0         0


In [37]:
#calcular para cada classe

certos = (df['classe'] == df['Previsto']).sum()
total = len(df)
print(f'Certos: {certos} de {total}')
print(f'Accuracy: {certos/total:.2%}')

Certos: 6191 de 6191
Accuracy: 100.00%
